In [6]:
# Getting ourworking directory
import os 
os.getcwd()

'c:\\Users\\HP\\Desktop\\VS_Code\\projects\\RAG_model\\RAG_for_Investment_evaluation\\research'

In [7]:
# Changing working directory
os.chdir("../")

os.getcwd()

'c:\\Users\\HP\\Desktop\\VS_Code\\projects\\RAG_model\\RAG_for_Investment_evaluation'

In [9]:
from dotenv import load_dotenv

load_dotenv()

os.environ["HF_TOKEN"] = os.getenv("HF_TOKEN")
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [10]:
from langchain_groq import ChatGroq 

llama4 = "meta-llama/llama-4-scout-17b-16e-instruct"
gemma = "gemma2-9b-it"
deepseek= "deepseek-r1-distill-llama-70b"

llama_model = ChatGroq(model=llama4)
llama_model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000028A17EB0890>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000028A17EA4110>, model_name='meta-llama/llama-4-scout-17b-16e-instruct', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [13]:
print(llama_model.invoke("Hi, can you explain about self attention in transformer architecture?").content)

Self-attention is a key component of the Transformer architecture, introduced in the paper "Attention Is All You Need" by Vaswani et al. in 2017. It's a mechanism that allows the model to attend to different parts of the input sequence simultaneously and weigh their importance.

**What is self-attention?**

In traditional recurrent neural networks (RNNs), the model processes the input sequence sequentially, using the previous hidden state to inform the next step. In contrast, self-attention allows the model to process the entire input sequence in parallel and compute the representation of each element (e.g., a word or token) by attending to all other elements in the sequence.

**How does self-attention work?**

The self-attention mechanism takes in a set of input vectors, typically representing a sequence of tokens (e.g., words or characters). The input vectors are linearly transformed into three vectors:

1. **Query (Q)**: represents the context in which the attention is being applied

In [14]:
from pathlib import Path 

data_path = "docs/"

file_names = os.listdir(data_path)

doc_path_list = [data_path + file for file in file_names]
doc_path_list

['docs/Neolen Pitch.pdf', 'docs/Pitch Deck EvolveDev.io .pdf']

In [ ]:
# Loading our test PDF 

from langchain_community.document_loaders import UnstructuredPDFLoader 



loader = UnstructuredPDFLoader(file_path=data_path,mode="elements", )

chunks = loader.load_and_split()

Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Cannot set gray stroke color because /'P4' is an invalid float value
Cannot set gray non-stroke color because /'P4' is an invalid float value
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Cannot set gray stroke color because /'P17' is an invalid float value
Cannot set gray non-stroke color because /'P17' is an invalid float value
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Cannot set gray stroke color because /'P28' is an invalid float value
Cannot set gray non-stroke color because /'P28' is an invalid float va

In [25]:
len(chunks)

92

In [ ]:
set([str(type(element)) for element in chunks])

{"<class 'langchain_core.documents.base.Document'>"}

In [22]:
set(element.metadata["category"] for element in chunks)

{'Header', 'NarrativeText', 'Title', 'UncategorizedText'}

In [15]:
# Load data 
from langchain_community.document_loaders.text import TextLoader
from langchain_community.document_loaders import PyPDFLoader, Docx2txtLoader, WebBaseLoader 

docs = []

for doc_file in doc_path_list:

    doc_path = Path(doc_file) 
    try: 
        if doc_file.endswith(".pdf"):
            loader = PyPDFLoader(doc_path)
        elif doc_file.endswith(".docx"):
            loader = Docx2txtLoader(doc_path)
        elif doc_file.endswith(".txt") or doc_file.endswith(".md"):
            loader = TextLoader(doc_path)
        else:
            print(f"Document type {doc_path.type} not supported")
            continue

        docs.extend(loader.load())

    except Exception as e:
        print(f"Error occured during loading a document: \n{doc_path.name} -> {e} ")

# # Load URL 
# url = ""
# try:
#     loader = WebBaseLoader(url)
#     docs.extend(loader.load())
# except Exception as e:
#     print(f"Error loading a URL: {url} -> {e} ")

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [16]:
docs

[Document(metadata={'producer': '3-Heights™ PDF Optimization API 6.27.2.6 (http://www.pdf-tools.com)', 'creator': 'Chromium', 'creationdate': '2025-01-28T07:04:53+00:00', 'moddate': '2025-01-28T07:05:02+00:00', 'source': 'docs\\Neolen Pitch.pdf', 'total_pages': 11, 'page': 0, 'page_label': '1'}, page_content='NEOLEN\nWE ENGINEER REVENUE-DRIVENPRODUCTS FOR MARKETSUCCESS.\nOVER MILLION LINES OF CODE |\nIMPACTING MILLIONS OF USERS |\nGENERATING MILLIONS IN REVENUE\nACROSS 10+ INDUSTRIES | EMPOWERING\n20+ SUCCESS STORIES | EXEMPLARY\nWORLD-CLASS ENGINEERING'),
 Document(metadata={'producer': '3-Heights™ PDF Optimization API 6.27.2.6 (http://www.pdf-tools.com)', 'creator': 'Chromium', 'creationdate': '2025-01-28T07:04:53+00:00', 'moddate': '2025-01-28T07:05:02+00:00', 'source': 'docs\\Neolen Pitch.pdf', 'total_pages': 11, 'page': 1, 'page_label': '2'}, page_content="Neolen is an award-winning IT Development Agency, team based\nin Delhi, India. Trusted by big companies and growing startups.\

In [17]:
# Split docs 
from langchain.text_splitter import RecursiveCharacterTextSplitter

test_splitter = RecursiveCharacterTextSplitter(
                    chunk_size = 5000, 
                    chunk_overlap = 500
)

doc_chunks = test_splitter.split_documents(docs)

In [18]:
from langchain_huggingface import HuggingFaceEmbeddings

model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
hf_embedding = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

c:\Users\HP\Desktop\VS_Code\projects\RAG_model\RAG_for_Investment_evaluation\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [22]:
# Tokenize and load document ino vector stores

from langchain_community.vectorstores import Chroma

vector_db = Chroma.from_documents(documents = doc_chunks, embedding = hf_embedding)

In [ ]:
vector_db.similarity_search("karan oberoi",)

[Document(metadata={'page': 8, 'creator': 'Chromium', 'moddate': '2025-01-28T07:05:02+00:00', 'total_pages': 11, 'creationdate': '2025-01-28T07:04:53+00:00', 'source': 'docs\\Neolen Pitch.pdf', 'page_label': '9', 'producer': '3-Heights™ PDF Optimization API 6.27.2.6 (http://www.pdf-tools.com)'}, page_content="KARAN OBEROI\nWHORV\nMRINALINI KHUSAPE\nPAWPURRFECT\nHARIENDERPREET SINGH\nLISTEN MONSTER \nI love Neolen. I would also like to\nsay thanks to all your staff.\nThanks for the great journey, it\nwas amazing collaborating with\nyou guys.\nNeolen's team provided us with\nexceptional IT support for our business.\nTheir quick response times and technical\nexpertise made all the difference for us.\nHighly recommend!\nI have been working with Neolen\nfor over a year now and I am\nextremely satis\x00ed with their\nservice.\nListen to Few Our Happy Clients"),
 Document(metadata={'creator': 'Chromium', 'page': 4, 'page_label': '5', 'moddate': '2025-01-28T07:05:02+00:00', 'source': 'docs\\Ne

In [19]:
# Retrieve
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains import create_history_aware_retriever, create_retrieval_chain

def _get_context_retriever_chain(vector_db, llm):
    retriever = vector_db.as_retriever()
    prompt = ChatPromptTemplate.from_messages([
        MessagesPlaceholder(variable_name="messages"),
        ("user", "{input}"),
        ("user", "Given the above conversation, generate a search query to look up in order to get inforamtion relevant to the conversation, focusing on the most recent messages."),
    ])
    retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

    return retriever_chain

In [20]:
from langchain.chains.combine_documents import create_stuff_documents_chain

def get_conversational_rag_chain(llm):
    retriever_chain = _get_context_retriever_chain(vector_db, llm)

    prompt = ChatPromptTemplate.from_messages([
        ("system",
        """You are a helpful assistant. You will have to answer to user's queries.
        You will have some context to help with your answers, but now always would be completely related or helpful.
        You can also use your knowledge to assist answering the user's queries.\n
        {context}"""),
        MessagesPlaceholder(variable_name="messages"),
        ("user", "{input}"),
    ])
    stuff_documents_chain = create_stuff_documents_chain(llm, prompt)

    return create_retrieval_chain(retriever_chain, stuff_documents_chain)

In [23]:
from langchain_core.messages import AIMessage, HumanMessage

llm_stream = ChatGroq(model = deepseek,
                      temperature=0.3, streaming=True)

messages = [
    {"role": "user", "content": "Hi"},
    {"role": "assistant", "content": "Hi there! How can I assist you today?"},
    {"role": "user", "content": "What is Neolen all about?"},
]
messages = [HumanMessage(content=m["content"]) if m["role"] == "user" else AIMessage(content=m["content"]) for m in messages]

conversation_rag_chain = get_conversational_rag_chain(llm_stream)
response_message = "*(RAG Response)*\n"
for chunk in conversation_rag_chain.pick("answer").stream({"messages": messages[:-1], "input": messages[-1].content}):
    response_message += chunk
    print(chunk, end="", flush=True)

messages.append({"role": "assistant", "content": response_message})

<think>
Alright, the user is asking, "What is Neolen all about?" I need to provide a clear and concise answer based on the context they've given. Let me go through the information provided.

First, Neolen is an award-winning IT Development Agency based in Delhi, India. They've been around since 2018, so they have over six years of experience. They've worked with 20+ happy clients across various industries, which shows their versatility and reliability.

They focus on engineering revenue-driven products, which means they don't just build software but ensure it's successful in the market. Their services include AI Agents Development, where they've built over 20 AI agents in three months. That's impressive and shows their expertise in AI.

They also have experience in ML models, particularly in areas like Audio, NLP, Predictive, and Computer Vision. They've built four plus ML models over three years, which indicates a strong foundation in machine learning.

App development is another key 

In [29]:
llm_stream = ChatGroq(model = deepseek,
                      temperature=0.3, streaming=True)

messages = [
    {"role": "user", "content": "Hi"},
    {"role": "assistant", "content": "Hi there! How can I assist you today?"},
    {"role": "user", "content": "At least5 additional insights you think are useful for the investors"},
]
messages = [HumanMessage(content=m["content"]) if m["role"] == "user" else AIMessage(content=m["content"]) for m in messages]

conversation_rag_chain = get_conversational_rag_chain(llm_stream)
response_message = "*(RAG Response)*\n"
for chunk in conversation_rag_chain.pick("answer").stream({"messages": messages[:-1], "input": messages[-1].content}):
    response_message += chunk
    print(chunk, end="", flush=True)

messages.append({"role": "assistant", "content": response_message})

<think>
Okay, so the user provided a lot of context about a company, including their core team and some product features. They asked for at least five additional insights that would be useful for investors. 

First, I need to figure out what investors care about. They usually look for growth potential, market fit, competitive advantage, strong teams, and clear financials. 

Looking at the context, the company has a solid team with a good mix of experience in various fields like blockchain, AI/ML, and cloud expertise. That's a plus because investors like experienced teams. 

The product seems to focus on actionable insights and improving development workflows, which is a hot area right now. Companies are always looking to optimize their processes, so that's a good market fit.

I should think about what else would make this company attractive. Maybe market expansion opportunities? If they can show they can enter new industries or geographies, that's a big plus.

Customer acquisition stra

In [30]:
llm_stream = ChatGroq(model = deepseek,
                      temperature=0.3, streaming=True)

messages = [
    {"role": "user", "content": "Hi"},
    {"role": "assistant", "content": "Hi there! How can I assist you today?"},
    {"role": "user", "content": "What are the key risk factors investors can consider from the pitch deck?"},
]
messages = [HumanMessage(content=m["content"]) if m["role"] == "user" else AIMessage(content=m["content"]) for m in messages]

conversation_rag_chain = get_conversational_rag_chain(llm_stream)
response_message = "*(RAG Response)*\n"
for chunk in conversation_rag_chain.pick("answer").stream({"messages": messages[:-1], "input": messages[-1].content}):
    response_message += chunk
    print(chunk, end="", flush=True)

messages.append({"role": "assistant", "content": response_message})

<think>
Alright, the user is asking about the key risk factors investors might consider from the provided pitch deck. Let me start by understanding the context. The pitch deck talks about a company with a specific product and a team. Investors would look for risks in several areas.

First, the market and competition. The pitch mentions the product helps with team progress and communication, but it doesn't specify the market size or competition. Investors would want to know if the market is big enough and how the product stands out.

Next, the technology. The product has features like automated reporting and customizable charts. Investors would assess if the tech is scalable and secure, especially since it's integrating with platforms like Slack and email.

Looking at the team, the founders have varied experience, which is a strength. However, the CEO has only 6+ years of experience, which might raise concerns about handling rapid scaling or complex challenges.

Customer acquisition is 